--- 
# Sandbox (Faune)
Tests for each part of this project

---

### Imports

In [1]:
#external libraries
import polars as pl
from datetime import date
import nest_asyncio

#internal libraries
from prompts import process_in_batches, import_data

## Importing data test

In [3]:
titles = import_data()

## Prompting test

### Basic sentences
Adapted from the mistral website

In [9]:
# testing on half of the dataset first
titles = titles.filter(pl.col('date').ge(date(2024,1,1)))

In [10]:
prompts = titles.select('prompts').to_series().to_list()

nest_asyncio.apply()

async for batch in process_in_batches(prompts, batch_size=1000, concurrency=50):
    batch_df = pl.DataFrame({
        "index": [r[0] for r in batch],
        "prompt": [r[1] for r in batch],
        "response": [r[2] for r in batch],
    })
    batch_df.write_parquet("responses.parquet")
    print(f"Saved batch of {len(batch)} rows.")



Processing prompts 0 to 999...
Saved batch of 1000 rows.
Processing prompts 1000 to 1999...
Saved batch of 1000 rows.
Processing prompts 2000 to 2999...
Saved batch of 1000 rows.
Processing prompts 3000 to 3999...
Saved batch of 1000 rows.
Processing prompts 4000 to 4999...
Saved batch of 1000 rows.
Processing prompts 5000 to 5999...
Saved batch of 1000 rows.
Processing prompts 6000 to 6999...
Saved batch of 1000 rows.
Processing prompts 7000 to 7999...
Saved batch of 1000 rows.
Processing prompts 8000 to 8999...
Saved batch of 1000 rows.
Processing prompts 9000 to 9999...
Saved batch of 1000 rows.
Processing prompts 10000 to 10999...
Saved batch of 1000 rows.
Processing prompts 11000 to 11641...
Saved batch of 642 rows.


In [13]:
pl.read_parquet('responses.parquet')

index,prompt,response
i64,str,str
11000,""" Here is a piece of news: …","""[ERROR] API error occurred: St…"
11001,""" Here is a piece of news: …","""{uncertain:1.0} {confidence (0…"
11002,""" Here is a piece of news: …","""{uncertain}: {0.5}: {0.25}: Th…"
11003,""" Here is a piece of news: …","""{increase}: {confidence (0.6)}…"
11004,""" Here is a piece of news: …","""{increase}: {confidence (0.4)}…"
…,…,…
11637,""" Here is a piece of news: …","""[ERROR] API error occurred: St…"
11638,""" Here is a piece of news: …","""[ERROR] API error occurred: St…"
11639,""" Here is a piece of news: …","""[ERROR] API error occurred: St…"
